## import pkg

In [1]:
# import sktime

### for preprocessing, we have different methods for different column (horizontal)

In [5]:
from sklearn.compose import ColumnTransformer

### for model training, we have step-by-step methods (vertical)

In [6]:
from sklearn.pipeline import make_pipeline

### for Nan Numerical data in dp, i use KNNImputer

In [7]:
# from sklearn.impute import KNNImputer

### for Nan categorical data, i use sklearn.impute.SimpleImputer

In [3]:
from sklearn.impute import SimpleImputer

### for categorical data, i use pd.series.astype("category")

df = pd.DataFrame({"A": list("abca"), "B": list("bccd")})
convert just columns "a" and "b"
df[["a", "b"]] = df[["a", "b"]].apply(pd.to_numeric)

### for numerical data, i use stdscaler

In [4]:
from sklearn.preprocessing import StandardScaler

## load dataset

In [2]:
path = "D:/202209/tbrain/訓練資料集_first/"
import os
os.chdir(path)
# import glob
# print(glob.glob(os.path.join(path, "*.csv")))
## 備註
## 這邊利用 os.path.join 來連接指定字串形成路徑，因為不同系統下的分隔符號可能不同

In [1]:
import pandas as pd

In [72]:
ccba = pd.read_csv("public_train_x_ccba_full_hashed.csv")
cdtx = pd.read_csv("public_train_x_cdtx0001_full_hashed.csv")
custinfo = pd.read_csv("public_train_x_custinfo_full_hashed.csv")
dp = pd.read_csv("public_train_x_dp_full_hashed.csv")
remit = pd.read_csv("public_train_x_remit1_full_hashed.csv")
alert_public = pd.read_csv("public_x_alert_date.csv")
alert_train = pd.read_csv("train_x_alert_date.csv")
y = pd.read_csv("train_y_answer.csv")

## prepare sub-dataset

In [13]:
import numpy as np

In [76]:
imp_occu = SimpleImputer(strategy='constant', fill_value=40)
custinfo[['occupation_code']] = imp_occu.fit_transform(custinfo[['occupation_code']])

In [77]:
imp_fisc = SimpleImputer(strategy='constant', fill_value=30)
dp[['fiscTxId']] = imp_occu.fit_transform(dp[['fiscTxId']])

In [78]:
imp_branch = SimpleImputer(strategy='constant', fill_value=400)
dp[['txbranch']] = imp_occu.fit_transform(dp[['txbranch']])

### transfer the categorical datas into pd.Categorical

In [80]:
ccba['cust_id'] = pd.Categorical(ccba['cust_id'],  categories = list(ccba['cust_id'].unique()))
cdtx['cust_id'] = pd.Categorical(cdtx['cust_id'],  categories = list(cdtx['cust_id'].unique()))
custinfo['cust_id'] = pd.Categorical(custinfo['cust_id'],  categories = list(custinfo['cust_id'].unique()))
dp['cust_id'] = pd.Categorical(dp['cust_id'],  categories = list(dp['cust_id'].unique()))
remit['cust_id'] = pd.Categorical(remit['cust_id'],  categories = list(remit['cust_id'].unique()))

In [79]:
ccba['byymm'] = pd.Categorical(ccba['byymm'],  categories = list(ccba['byymm'].unique()), ordered=True)

cdtx['date'] = pd.Categorical(cdtx['date'],  categories = list(cdtx['date'].unique()), ordered=True)
cdtx['country'] = pd.Categorical(cdtx['country'],  categories = list(cdtx['country'].unique()))
cdtx['cur_type'] = pd.Categorical(cdtx['cur_type'],  categories = list(cdtx['cur_type'].unique()))

custinfo['risk_rank'] = pd.Categorical(custinfo['risk_rank'], categories = list(custinfo['risk_rank'].unique()), ordered=True)
custinfo['AGE'] = pd.Categorical(custinfo['AGE'], categories = list(custinfo['AGE'].unique()), ordered=True)
custinfo['occupation_code'] = pd.Categorical(custinfo['occupation_code'], categories = list(custinfo['occupation_code'].unique()))

dp['tx_date'] = pd.Categorical(dp['tx_date'], categories = list(dp['tx_date'].unique()), ordered=True)
dp['tx_time'] = pd.Categorical(dp['tx_time'], categories = list(dp['tx_time'].unique()), ordered=True)
dp['debit_credit'] = pd.Categorical(dp['debit_credit'], categories = list(dp['debit_credit'].unique()))
dp['tx_type'] = pd.Categorical(dp['tx_type'], categories = list(dp['tx_type'].unique()))
dp['info_asset_code'] = pd.Categorical(dp['info_asset_code'], categories = list(dp['info_asset_code'].unique()))
dp['fiscTxId'] = pd.Categorical(dp['fiscTxId'], categories = list(dp['fiscTxId'].unique()))
dp['txbranch'] = pd.Categorical(dp['txbranch'], categories = list(dp['txbranch'].unique()))
dp['cross_bank'] = pd.Categorical(dp['cross_bank'], categories = list(dp['cross_bank'].unique()))
dp['ATM'] = pd.Categorical(dp['ATM'], categories = list(dp['ATM'].unique()))

remit['trans_date'] = pd.Categorical(remit['trans_date'], categories = list(remit['trans_date'].unique()), ordered=True)
remit['trans_no'] = pd.Categorical(remit['trans_no'], categories = list(remit['trans_no'].unique()))

## prepare dp

In [81]:
imputer_branch = SimpleImputer(strategy='median')

dp["tx_amt"] = imputer_branch.fit_transform(dp[["tx_amt"]])

In [82]:
# calculate into NTD
dp['NTD'] = dp['tx_amt']*dp['exchg_rate'].round(2)

# is it originally used NTD?
dp['isNTD'] = dp.loc[dp["exchg_rate"] == 1.0, "isNTD"] = 1
dp['isNTD'] = dp.loc[dp["exchg_rate"] != 1.0, "isNTD"] = 0

dp[dp['exchg_rate']!=1.0].head()

,cust_id,debit_credit,tx_date,tx_time,tx_type,tx_amt,exchg_rate,info_asset_code,fiscTxId,txbranch,cross_bank,ATM,NTD,isNTD
199,01b1735c3dbf0a35c1c989bb45434ce4326b365b73db06...,CR,230,14,2,3715.0,27.83,20,40.0,40.0,0,1,103388.45,0
200,01b1735c3dbf0a35c1c989bb45434ce4326b365b73db06...,CR,231,9,2,5029.0,27.80,20,40.0,40.0,0,1,139806.20,0
201,01b1735c3dbf0a35c1c989bb45434ce4326b365b73db06...,DB,231,14,2,3715.0,27.84,20,40.0,40.0,0,1,103425.60,0
202,01b1735c3dbf0a35c1c989bb45434ce4326b365b73db06...,DB,231,14,2,8428.0,27.84,20,40.0,40.0,0,1,234635.52,0
205,01b1735c3dbf0a35c1c989bb45434ce4326b365b73db06...,DB,291,11,2,2064.0,27.63,20,40.0,40.0,0,1,57028.32,0


In [25]:
# booooom
# from sklearn.impute import KNNImputer

# imputer = KNNImputer(n_neighbors=2, weights="uniform")

# dp["tx_amt"] = imputer.fit_transform(dp[["tx_amt"]]).ravel()


## standardize

In [83]:
standardizer = StandardScaler()

ccba[['lupay', 'cycam', 'usgam', 'clamt', 'csamt', 'inamt', 'cucsm', 'cucah']] = standardizer.fit_transform(ccba[['lupay', 'cycam', 'usgam', 'clamt', 'csamt', 'inamt', 'cucsm', 'cucah']])

cdtx[['amt']] = standardizer.fit_transform(cdtx[['amt']])

dp[['NTD']]  = standardizer.fit_transform(dp[['NTD']])

remit[['trade_amount_usd']] = standardizer.fit_transform(remit[['trade_amount_usd']])


## Prepare training/testing target

In [111]:
y_train = y.join(alert_train.set_index('alert_key'), on='alert_key')
y_train = y_train.join(custinfo[['alert_key', 'cust_id']].set_index('alert_key'), on='alert_key', how='inner')
y_train = y_train[['sar_flag', 'alert_key','cust_id']]
print(y_train.shape)
y_train.head()

(23906, 3)


,sar_flag,alert_key,cust_id
0,0,171189,972ee157e63316e8a50dd489bc93730a3ee8a8959d5c6b...
1,0,171202,a10ab33f90926fb18d7bb5e78034d7f04a1fbed95b7951...
2,0,171599,3433ecc068ed1c9e2f5037cab5d42d7b901e9bd624c1fb...
3,0,171737,a0861608615a9365d90f4ba0a813c0ea0471987f925c8b...
4,0,171142,a39fea9aec90969fe66a2b2b4d1b86368a2d38e8b8d4bf...


In [ ]:
#alert_join = pd.concat([alert_train, alert_public],axis=0)
#alert_join = alert_join.join(custinfo.set_index('alert_key'), on='alert_key')
#alert_join.iloc[23900:23910]

,alert_key,date,cust_id,risk_rank,occupation_code,total_asset,AGE
23900,351871,364,7b87ff62b8c40d8be12f571e2243cffadcb2501a57b8c8...,1,17.0,39100.0,2
23901,352132,364,c49b33d1fde790ec03584672903f296e486615adfdc989...,1,19.0,3218731.0,3
23902,352125,364,0c2dc5fedc3689abf5ff4be14fe8fea8d23d22068297c2...,3,19.0,928963.0,3
23903,352080,364,1a93961c8fa830a1f32f5316b44f4964d65397f0311b11...,2,9.0,69080.0,6
23904,352075,364,9586b80d3ad1d74c0a1efe792cae2ecd073243fb01ed33...,3,3.0,262604.0,6
23905,351927,364,7dc5256ef411c745dc3064da2f231c104ca270f564d3d0...,1,9.0,332834.0,2
0,352342,365,7629f010080267f680a4ca9c6cac71c4d4d86933dcc7ed...,1,17.0,506366.0,3
1,352866,365,e50deb3f2f87bf45e04940303f80c96929c836b0b03f54...,1,12.0,309830.0,4
2,352696,365,c6def618ad861703c025be4f41bdf7569310228ae93aef...,1,2.0,444392.0,3
3,352330,365,96d70b798f211393cfbcb83a8acd19f3e8c9c5c205960a...,1,19.0,143930.0,4


In [112]:
y_test = alert_public
y_test = y_test.join(custinfo[['alert_key', 'cust_id']].set_index('alert_key'), on='alert_key', how='inner')
y_test['sar_flag'] = ""
y_test = y_test[['sar_flag', 'alert_key','cust_id']]
y_test.head()

,sar_flag,alert_key,cust_id
0,,352342,7629f010080267f680a4ca9c6cac71c4d4d86933dcc7ed...
1,,352866,e50deb3f2f87bf45e04940303f80c96929c836b0b03f54...
2,,352696,c6def618ad861703c025be4f41bdf7569310228ae93aef...
3,,352330,96d70b798f211393cfbcb83a8acd19f3e8c9c5c205960a...
4,,352683,40fa92acb89d1ebb319f0f200d31a5fcbdfcbd8ad432d7...


# bcuz sar is imbalance, i use `imblearn`

In [114]:
train_dp = y_train.join(dp.set_index('cust_id'), on='cust_id')
train_dp.head()

,sar_flag,alert_key,cust_id,debit_credit,tx_date,tx_time,tx_type,tx_amt,exchg_rate,info_asset_code,fiscTxId,txbranch,cross_bank,ATM,NTD,isNTD
0,0,171189,972ee157e63316e8a50dd489bc93730a3ee8a8959d5c6b...,CR,282,12,2,26198.0,27.740,20,40.0,40.0,0,1,0.017923,0.0
0,0,171189,972ee157e63316e8a50dd489bc93730a3ee8a8959d5c6b...,CR,264,0,2,52372.0,1.000,16,40.0,40.0,0,0,-0.033177,0.0
0,0,171189,972ee157e63316e8a50dd489bc93730a3ee8a8959d5c6b...,CR,32,19,2,46411.0,27.955,20,40.0,40.0,0,1,0.061185,0.0
0,0,171189,972ee157e63316e8a50dd489bc93730a3ee8a8959d5c6b...,CR,151,15,2,460812.0,27.795,20,40.0,40.0,0,1,0.933588,0.0
0,0,171189,972ee157e63316e8a50dd489bc93730a3ee8a8959d5c6b...,CR,39,9,2,52379.0,27.765,20,40.0,40.0,0,1,0.073036,0.0


In [ ]:
# import library
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(random_state=42, replacement=True)# fit predictor and target variable
x_rus, y_rus = rus.fit_resample( train_dp.loc[:, train_dp.columns != 'sar_flag'], train_dp['sar_flag'])

print('original dataset shape:', Counter(y_train['sar_flag']))
print('Resample dataset shape', Counter(y_rus))

# Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:

modelg=GaussianNB()
modelg.fit(X_train,y_train)
modelg.predict(X_test)